In [10]:
import os
import glob
import rasterio
import numpy as np
import re # For using regular expressions to extract scene IDs

def calculate_ndvi(red_file_path, nir_file_path, output_file_path):
    """
    Calculates NDVI from Red and NIR band files and saves the result.
    (This function remains largely the same as the previous rasterio version,
     assuming input Red and NIR bands need scaling if they are raw Landsat values)
    """
    try:
        # --- Define Landsat Collection 2 Level 2 scaling factors ---
        # If your "Red_band(...)" files are already reflectance (0-1),
        # set scale_factor = 1.0 and offset = 0.0
        # If they are raw scaled integers from Landsat SR products:
        scale_factor = 0.0000275
        offset = -0.2
        
        with rasterio.open(red_file_path) as red_src:
            red_profile = red_src.profile
            red_int = red_src.read(1).astype('float32')
            red = (red_int * scale_factor) + offset
            if red_src.nodata is not None:
                red[red_int == red_src.nodata] = np.nan

        with rasterio.open(nir_file_path) as nir_src:
            nir_int = nir_src.read(1).astype('float32')
            nir = (nir_int * scale_factor) + offset
            if nir_src.nodata is not None:
                nir[nir_int == nir_src.nodata] = np.nan
        
        np.seterr(divide='ignore', invalid='ignore')
        numerator = nir - red
        denominator = nir + red
        ndvi = numerator / denominator
        
        output_nodata = -9999.0
        ndvi[np.isinf(ndvi)] = output_nodata
        ndvi[np.isnan(ndvi)] = output_nodata
        ndvi[denominator == 0] = output_nodata
        
        valid_ndvi_mask = (ndvi >= -1) & (ndvi <= 1)
        ndvi[~valid_ndvi_mask & (ndvi != output_nodata)] = output_nodata

        profile = red_profile
        profile.update(
            dtype=rasterio.float32,
            count=1,
            compress='lzw',
            nodata=output_nodata
        )
        
        with rasterio.open(output_file_path, 'w', **profile) as dst:
            dst.write(ndvi.astype(rasterio.float32), 1)
            
        print(f"Successfully calculated NDVI and saved to: {output_file_path}")

    except Exception as e:
        print(f"Error processing {os.path.basename(red_file_path)} or {os.path.basename(nir_file_path)}: {e}")

def find_band_pairs_custom(input_dir, 
                           red_filename_keyword="Red_band", 
                           nir_filename_keyword="Nir_band",
                           red_band_id_suffix="_SR_B4", 
                           nir_band_id_suffix="_SR_B5",
                           file_extension=".TIF"):
    """
    Finds pairs of Red and NIR band files based on custom keywords, 
    scene IDs in parentheses, and band identifier suffixes.
    """
    pairs = []
    # Regex to extract the scene ID like (147-041) from filenames
    # It looks for content within parentheses: \( (.*?) \)
    scene_id_regex = re.compile(r'\((.*?)\)') # (.*?) is a non-greedy match

    potential_files = {} # Dictionary to store files found, keyed by scene_id

    # Search recursively for TIF files (case-insensitive extension)
    search_pattern_lower = os.path.join(input_dir, '**', f'*{file_extension.lower()}')
    search_pattern_upper = os.path.join(input_dir, '**', f'*{file_extension.upper()}')
    
    all_tif_files = glob.glob(search_pattern_lower, recursive=True) + \
                    glob.glob(search_pattern_upper, recursive=True)
    all_tif_files = list(set(all_tif_files)) # Remove duplicates if any

    print(f"Found {len(all_tif_files)} files with extension '{file_extension}' in '{input_dir}' (and subdirectories).")
    if not all_tif_files:
        return pairs

    for f_path in all_tif_files:
        filename = os.path.basename(f_path)
        
        # Try to extract scene_id (content within parentheses)
        scene_id_match = scene_id_regex.search(filename)
        if scene_id_match:
            scene_id = scene_id_match.group(1) # e.g., "147-041"
            
            if scene_id not in potential_files:
                potential_files[scene_id] = {}
            
            # Check if it's a Red band
            if red_filename_keyword in filename and filename.endswith(red_band_id_suffix + file_extension.upper()) \
            or red_filename_keyword in filename and filename.endswith(red_band_id_suffix + file_extension.lower()):
                if 'red' in potential_files[scene_id]:
                    print(f"Warning: Multiple Red files found for scene_id ({scene_id}). Overwriting {potential_files[scene_id]['red']} with {f_path}")
                potential_files[scene_id]['red'] = f_path
            
            # Check if it's a NIR band
            elif nir_filename_keyword in filename and filename.endswith(nir_band_id_suffix + file_extension.upper()) \
            or nir_filename_keyword in filename and filename.endswith(nir_band_id_suffix + file_extension.lower()):
                if 'nir' in potential_files[scene_id]:
                    print(f"Warning: Multiple NIR files found for scene_id ({scene_id}). Overwriting {potential_files[scene_id]['nir']} with {f_path}")
                potential_files[scene_id]['nir'] = f_path
        # else:
            # print(f"Debug: Could not extract scene ID from filename: {filename}")


    # Now form pairs from the collected potential files
    for scene_id, bands in potential_files.items():
        if 'red' in bands and 'nir' in bands:
            pairs.append((bands['red'], bands['nir']))
            # print(f"Debug: Found pair for scene_id ({scene_id}): R={bands['red']}, N={bands['nir']}")
        else:
            print(f"Warning: Missing Red or NIR band for scene_id ({scene_id}). Found: {bands}")
            
    return pairs

# --- MAIN SCRIPT EXECUTION ---
if __name__ == "__main__":
    # --- USER: SET YOUR PATHS AND FILE NAMING CONVENTIONS HERE ---
    landsat_input_directory = "/Users/macbook/Documents/Geo_spatial_Project/Data/Landset Images /Ndvi_bands"
    ndvi_output_directory = "/Users/macbook/Documents/Geo_spatial_Project/Data/Landset Images/Ndvi_output/"
    
    # Keywords and suffixes based on your filenames like "Red_band(147-041)_SR_B4.TIF"
    # Ensure these match your *actual* filenames precisely.
    red_filename_keyword = "Red_band"
    nir_filename_keyword = "Nir_band"
    red_band_id_suffix = "_SR_B4"    # The part just before the .TIF extension for Red
    nir_band_id_suffix = "_SR_B5"    # The part just before the .TIF extension for NIR
    file_extension = ".TIF"          # Or ".tif" if your files use lowercase
    # --- END OF USER SETTINGS ---

    # Create output directory if it doesn't exist
    if not os.path.exists(ndvi_output_directory):
        os.makedirs(ndvi_output_directory)
        print(f"Created output directory: {ndvi_output_directory}")

    # Find all Red and NIR band pairs based on the custom naming convention
    band_pairs = find_band_pairs_custom(landsat_input_directory, 
                                       red_filename_keyword, 
                                       nir_filename_keyword,
                                       red_band_id_suffix, 
                                       nir_band_id_suffix,
                                       file_extension)

    if not band_pairs:
        print(f"\nNo Red and NIR band pairs found in '{landsat_input_directory}'.")
        print("Please check:")
        print("1. Your `landsat_input_directory` path.")
        print("2. Your actual filenames and ensure they consistently follow the pattern:")
        print(f"   '{red_filename_keyword}(SCENE_ID){red_band_id_suffix}{file_extension}' for Red bands")
        print(f"   '{nir_filename_keyword}(SCENE_ID){nir_band_id_suffix}{file_extension}' for NIR bands")
        print(f"   (e.g., Red_band(123-456)_SR_B4.TIF and Nir_band(123-456)_SR_B5.TIF)")
        print("3. Correct any typos in your filenames (like the extra dot you mentioned).")
        print("4. Ensure the `file_extension` variable matches your files (.TIF or .tif).")
    else:
        print(f"\nFound {len(band_pairs)} Red/NIR band pairs to process.")

        for i, (red_path, nir_path) in enumerate(band_pairs):
            print(f"\nProcessing pair {i+1}/{len(band_pairs)}:")
            # print(f"  Red: {red_path}") # Already printed by find_band_pairs_custom if debug is on
            # print(f"  NIR: {nir_path}")
            
            # Define output filename for the NDVI raster
            # Extracts the scene_id and common parts for a more descriptive output name
            red_basename = os.path.basename(red_path)
            scene_id_match = re.search(r'\((.*?)\)', red_basename)
            scene_id_str = scene_id_match.group(1) if scene_id_match else "UNKNOWN_SCENE"
            
            output_ndvi_file = os.path.join(ndvi_output_directory, f"NDVI_({scene_id_str}).TIF")
            
            calculate_ndvi(red_path, nir_path, output_ndvi_file)
            
    print("\nNDVI processing complete.")

Found 18 files with extension '.TIF' in '/Users/macbook/Documents/Geo_spatial_Project/Data/Landset Images /Ndvi_bands' (and subdirectories).

Found 9 Red/NIR band pairs to process.

Processing pair 1/9:
Successfully calculated NDVI and saved to: /Users/macbook/Documents/Geo_spatial_Project/Data/Landset Images/Ndvi_output/NDVI_(148-041).TIF

Processing pair 2/9:
Successfully calculated NDVI and saved to: /Users/macbook/Documents/Geo_spatial_Project/Data/Landset Images/Ndvi_output/NDVI_(149-041).TIF

Processing pair 3/9:
Successfully calculated NDVI and saved to: /Users/macbook/Documents/Geo_spatial_Project/Data/Landset Images/Ndvi_output/NDVI_(149-042).TIF

Processing pair 4/9:
Successfully calculated NDVI and saved to: /Users/macbook/Documents/Geo_spatial_Project/Data/Landset Images/Ndvi_output/NDVI_(148-042).TIF

Processing pair 5/9:
Successfully calculated NDVI and saved to: /Users/macbook/Documents/Geo_spatial_Project/Data/Landset Images/Ndvi_output/NDVI_(147-043).TIF

Processing pa